In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 글꼴깨짐 방지
import matplotlib
import matplotlib.font_manager as fm

import warnings

# 경고 메시지를 무시하고 숨기거나
warnings.filterwarnings(action='ignore')

fm.get_fontconfig_fonts()
font_location = 'C:/Windows/Fonts/malgun.ttf' # For Windows
font_name = fm.FontProperties(fname=font_location).get_name()
matplotlib.rc('font', family=font_name)

In [18]:
df = pd.read_csv('train.csv')
df.head()

,단지코드,총세대수,임대건물구분,지역,공급유형,전용면적,전용면적별세대수,공가수,자격유형,임대보증금,임대료,도보 10분거리 내 지하철역 수(환승노선 수 반영),도보 10분거리 내 버스정류장 수,단지내주차면수,등록차량수
0,C2483,900,아파트,경상북도,국민임대,39.72,134,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
1,C2483,900,아파트,경상북도,국민임대,39.72,15,38.0,A,15667000,103680,0.0,3.0,1425.0,1015.0
2,C2483,900,아파트,경상북도,국민임대,51.93,385,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
3,C2483,900,아파트,경상북도,국민임대,51.93,15,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0
4,C2483,900,아파트,경상북도,국민임대,51.93,41,38.0,A,27304000,184330,0.0,3.0,1425.0,1015.0


In [19]:
def preprocessing(df):
    # 오류 단지코드가 존재하는 행들을  사전에 제거
    df_error =  ['C1095', 'C2051', 'C1218', 'C1894', 'C2483', 'C1502', 'C1988']
    #df_error =  ['C2335', 'C1327']
    df = df[~df['단지코드'].isin(df_error)].reset_index(drop=True)
    df.rename(columns = {'도보 10분거리 내 지하철역 수(환승노선 수 반영)':'지하철','도보 10분거리 내 버스정류장 수':'버스'},inplace=True)
    df.drop(columns=['임대보증금','임대료','자격유형','임대건물구분'],axis = 1,inplace=True)
    지역_비율 = (df.groupby(['지역'])['총세대수'].count())/(df.groupby(['지역'])['총세대수'].count().sum())*100
    지역_비율=지역_비율.reset_index(name='지역_비율')
    공급유형_비율 = (df.groupby(['공급유형'])['총세대수'].count())/(df.groupby(['공급유형'])['총세대수'].count().sum())*100
    공급유형_비율=공급유형_비율.reset_index(name='공급유형_비율')
    df = pd.merge(df,지역_비율, on='지역')
    df = pd.merge(df,공급유형_비율, on='공급유형')
    df.drop(columns=['지역','공급유형','단지코드'],axis = 1,inplace=True)
    df=df.dropna(axis=0)
    df = df[['총세대수', '전용면적', '전용면적별세대수', '공가수', '지하철', '버스', '단지내주차면수', '공급유형_비율',
       '지역_비율', '등록차량수']]
    return df

In [20]:
df = preprocessing(df)
df

,총세대수,전용면적,전용면적별세대수,공가수,지하철,버스,단지내주차면수,공급유형_비율,지역_비율,등록차량수
0,545,33.48,276,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
1,545,39.60,60,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
2,545,39.60,20,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
3,545,46.90,38,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
4,545,46.90,19,17.0,0.0,3.0,624.0,59.944751,12.741713,205.0
...,...,...,...,...,...,...,...,...,...,...
2891,370,51.99,80,9.0,0.0,3.0,1590.0,0.310773,2.209945,1595.0
2892,370,59.93,142,9.0,0.0,3.0,1590.0,0.310773,2.209945,1595.0
2893,790,51.32,126,13.0,0.0,3.0,673.0,0.310773,2.209945,645.0
2894,790,59.88,49,13.0,0.0,3.0,673.0,0.310773,2.209945,645.0


In [5]:
# Splitting the data into train and test split
#from sklearn.model_selection import train_test_split
#X = df.iloc[:, 1:-1]
#y = df.iloc[:,-1]
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
import lightgbm as lgb

model = lgb.LGBMRegressor()
# model.fit(X_train, y_train)
# pred = model.predict(X_test)

In [7]:
# 스케일링 없이
from time import time
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

start = time()
model.fit(X_train, y_train)
train_time = time() - start
start = time()
y_pred = model.predict(X_test)
predict_time = time()-start    
print(model)
print("\tTraining time: %0.3fs" % train_time)
print("\tPrediction time: %0.3fs" % predict_time)
print("\tExplained variance:", explained_variance_score(y_test, y_pred))
print("\tMean absolute error:", mean_absolute_error(y_test, y_pred))
print("\tR2 score:", r2_score(y_test, y_pred))
print()

NameError: name 'X_train' is not defined

In [7]:
# 데이터 파일 스탠다드 스케일링
from sklearn.preprocessing import StandardScaler
standardScaler = StandardScaler()
print(standardScaler.fit(df))
train_data_standardScaled = standardScaler.transform(df)

StandardScaler()


In [10]:
# Splitting the data into train and test split

from sklearn.model_selection import train_test_split
sc_X = df.iloc[:, :-1]
y = df.iloc[:,-1]
sc_X_train, sc_X_test, y_train, y_test = train_test_split(sc_X, y, test_size=0.2, random_state=42)

In [12]:
#  스탠다드 스케일링
import lightgbm as lgb

sc_model = lgb.LGBMRegressor()
model.fit(sc_X_train, y_train)
sc_pred = model.predict(sc_X_test)

In [13]:
# 스케일링한 모델 평가 mse 0.8% 잡았음
from time import time
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

start = time()
sc_model.fit(sc_X_train, y_train)
train_time = time() - start
start = time()
y_pred = sc_model.predict(sc_X_test)
predict_time = time()-start    
print(sc_model)
print("\tsc_Training time: %0.3fs" % train_time)
print("\tsc_Prediction time: %0.3fs" % predict_time)
print("\tsc_Explained variance:", explained_variance_score(y_test, y_pred))
print("\tsc_Mean absolute error:", mean_absolute_error(y_test, y_pred))
print("\tsc_R2 score:", r2_score(y_test, y_pred))
print()

LGBMRegressor()
	sc_Training time: 0.145s
	sc_Prediction time: 0.004s
	sc_Explained variance: 0.9832310354572736
	sc_Mean absolute error: 32.621006109570274
	sc_R2 score: 0.9831380780340911



In [14]:
# 오이피클
import joblib

joblib.dump(model, 'lightgbm.pkl')

['lightgbm.pkl']

In [ ]:
# 핵심 파라미터들 넣기
params = {'learning_rate': (0.2,0.3,0.4), # 부스팅 각 이터레이션 마다 곱해지는 가중치로 모형 성능과 학습시간에 영향을 준다. 
          'max_depth': [8, 4, 2, 1, -1],# 최대 깊이
          'boosting': ['gbdt'], # ecision tree와 학습 데이터의 일부만 사용하여 부스팅을 수행한다.
          'application':['mean_absolute_error'],
          'min_data_in_leaf':[20], # left가 가지고 있는 최소한의 레코드. 디폴트 값은 20을 해결 할때 사용
          'objective': ['regression'], # 회귀
          'metric': ['mae'],# Mean absolute error
          'maximum_leaves':[31],# 트리의  최대 리프 수. 트리의 최대 깊이, 모형 성능, 학습속도에 영향을 미친다.
          'max_cat_group' :[64], # 카테고리의 수가 클때, 분기 포인트를 찾음
          'is_training_metric': [True],
          'column_sampling':[0.3],# 각 이터레이션에 사용되는 칼럼의 비율
          'num_leaves': [144], # 'maximum_leaves' 와 함께 사용하는 파라미터 
          'feature_fraction': [1,0.7,0.6],# 각 이터레이션에 사용되는 칼럼의 비율. 
          'bagging_fraction': [0.7,0.3,0.1 ],# 일반적으로 훈련 속도를 높이거나, 과적합을 피할때 사용한다.
          'bagging_freq': [5] # 각 이터레이션에 사용되는 행의 비율}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(sc_model,params, verbose=10)
sc_model = grid.fit(X,y)
print(sc_model.best_params_,'\n')
print(sc_model.best_estimator_,'\n')

In [ ]:
sc_params = sc_model.best_params_
sc_params

In [ ]:
scores_df = pd.DataFrame(grid.cv_results_)
scores_df

In [ ]:
grid.best_score_

In [ ]:
import lightgbm as lgb

In [ ]:
LGBMRegressor_model = lgb.LGBMRegressor(application= 'mean_absolute_error',
 bagging_fraction= 0.6,
 bagging_freq= 5,
 boosting= 'gbdt',
 feature_fraction= 0.9,
 is_training_metric= True,
 learning_rate= 0.1,
 max_cat_group= 64,
 max_depth= 4,
 metric= 'mae',
 min_data_in_leaf= 20,
 num_leaves= 144,
 objective= 'regression')
LGBMRegressor_model.fit(sc_X_train, y_train)
pred = LGBMRegressor_model.predict(sc_X_test)

In [ ]:
from sklearn.metrics import accuracy_score, mean_absolute_error
mean_absolute_error(y_test, pred)

In [ ]:
# 로버츠 스케일링(아웃라이어 최소화)
from sklearn.preprocessing import RobustScaler
rc = RobustScaler()
rc_X_train = rc.fit_transform(X_train)
rc_X_test = rc.transform(X_test)

In [ ]:
import lightgbm as lgb

rc_model = lgb.LGBMRegressor()
model.fit(rc_X_train, y_train)
rc_pred = model.predict(rc_X_test)

In [ ]:
# 로버츠스케일링한 모델평가 기준모델보다 mae는 0.7정도,R2 score는 0.006가량 높음
from time import time
from sklearn.metrics import explained_variance_score,mean_absolute_error,r2_score

start = time()
model.fit(rc_X_train, y_train)
train_time = time() - start
start = time()
y_pred = model.predict(rc_X_test)
predict_time = time()-start    
print('rc_model')
print("\tsc_Training time: %0.3fs" % train_time)
print("\tsc_Prediction time: %0.3fs" % predict_time)
print("\tsc_Explained variance:", explained_variance_score(y_test, y_pred))
print("\tsc_Mean absolute error:", mean_absolute_error(y_test, y_pred))
print("\tsc_R2 score:", r2_score(y_test, y_pred))
print()

In [ ]:
# best params 찾기
from sklearn.model_selection import GridSearchCV

rc_grid = GridSearchCV(rc_model,params, verbose=10)
rc_model = grid.fit(X,y)
print(rc_model.best_params_,'\n')
print(rc_model.best_estimator_,'\n')
print()

In [ ]:
# 똑같이 나옴
rc_params = rc_model.best_params_
rc_params

In [ ]:
# 결론 StandardScaler한 기존 모델이 가장 높은 정확도를 지녔다 